We will work today with 'momentum_data.xlsx' file. It contains several tabs:

- The 1st tab contains the momentum factor as an excess return: $\tilde{r}^{\text{mom}}$.

- The 3rd tab contains returns on portfolios corresponding to scored momentum deciles.

    - $r^{\text{mom(1)}}$ denotes the portfolio of stocks in the lowest momentum decile, the 'losers' with the lowest past returns.
    
    - $r^{\text{mom(10)}}$ denotes the portfolio of stocks in the highest momentum decile.
    
- The 4th tab gives portfolios sorted by momentum and size.

    - $r^{\text{momSU}}$ denotes the portfolio of small stocks in the top 3 deciles of momentum scores.
    
    - $r^{\text{momBD}}$ denotes the portfolio of big-stocks in the bottom 3 deciles of momentum scores.

### Task 1.

Check that momentum return, $\tilde{r}^{\text{mom:FF}}$, given in the second tab, is constructed:

$\tilde{r}^{\text{mom:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2-
(r^{\text{momBD}}+r^{\text{momSd}})/2$

In [103]:
import pandas as pd

In [104]:
#import data from tables: factors (excess returns), momentum (excess returns), size_sorts (total returns).
factors = pd.read_excel("momentum_data.xlsx", "factors (excess returns)", index_col=0)
momentum = pd.read_excel("momentum_data.xlsx", "momentum (excess returns)", index_col=0)
size_sorts = pd.read_excel("momentum_data.xlsx", "size_sorts (total returns)", index_col=0)

#Create a new Data Frame
r_mom_FF = pd.DataFrame(index=momentum.index)

#Add our indicators to Data Frame r_mom_f
r_mom_FF["UMD 1"] = round(momentum["UMD"], 1)
r_mom_FF["UMD 2"] = round((size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 \
                          - (size_sorts["BIG LoPRIOR"] + size_sorts["SMALL LoPRIOR"])/2, 1)

#Check if the momentum return 𝑟̃mom:FF, given in the second tab, is constructed: 
#                                                                  𝑟̃mom:FF=(𝑟momBU+𝑟momSU)/2−(𝑟momBD+𝑟momSd)/2
all(r_mom_FF["UMD 1"] == r_mom_FF["UMD 2"])

True

### Task 2

Fill in next table 1 with the appropriate stats for $\tilde{r}^{\text{mom:FF}}$. Please note, that $\tilde{r}^m$ is the excess market return, MKT; $\tilde{r}^v$ is the value portfolio, HML.

<img src='table1.png'>

In [105]:
#Create a list with years that we need to analize
indexes = ["1927-2022", "1927-1993", "1994-2008", "2009-2022"]

#Create a new Data Frame
momentum_per = pd.DataFrame()

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for each period of years
for data in indexes:
    momentum_per.loc[data, "Mean"] = r_mom_FF.loc[data[:4]:data[-4:], "UMD 1"].mean()
    momentum_per.loc[data, "Vol"] = r_mom_FF.loc[data[:4]:data[-4:], "UMD 1"].std()
    momentum_per.loc[data, "Sharpe"] = momentum_per.loc[data, "Mean"] / momentum_per.loc[data, "Vol"]
    momentum_per.loc[data, "Skewness"] = r_mom_FF.loc[data[:4]:data[-4:], "UMD 1"].skew()
    momentum_per.loc[data, "Corr_to_Rm"] = r_mom_FF.loc[data[:4]:data[-4:], "UMD 1"].corr(factors.loc[data[:4]:data[-4:], "MKT"])
    momentum_per.loc[data, "Corr_to_Rv"] = r_mom_FF.loc[data[:4]:data[-4:], "UMD 1"].corr(factors.loc[data[:4]:data[-4:], "HML"])

momentum_per

,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
1927-2022,0.000609,0.049822,0.012228,-2.188826,-0.345801,-0.405482
1927-1993,0.000871,0.047991,0.018142,-3.025223,-0.371853,-0.489162
1994-2008,0.003889,0.060134,0.064671,-0.482249,-0.250653,-0.109297
2009-2022,-0.004242,0.046000,-0.092226,-1.684882,-0.347623,-0.399363


### Task 3

Construct your own long-only implementation:

$\tilde{r}^{\text{momU:FF}} = (r^{\text{momBU}}+r^{\text{momSU}})/2 - r^f.$

Note that you have to subtract the risk-free rate (RF) to get the excess return of this portfolio.

Fill out Table 2 for the data in the period 1994-2022.

<img src='table2.png'>

In [106]:
#import data from tables: risk-free rate
risk_free_rate = pd.read_excel("momentum_data.xlsx", "risk-free rate", index_col=0)

#Create a new Data Frame
r_mom_u_ff = pd.DataFrame()

#Calculate indicator: R_momU_FF, and add indicator UMD to our Data Frame
r_mom_u_ff["R_momU_FF"] = (size_sorts["BIG HiPRIOR"] + size_sorts["SMALL HiPRIOR"])/2 - risk_free_rate["RF"]
r_mom_u_ff["UMD"] = momentum["UMD"].copy()

#Slice our data to the period of data: 1994-2022
r_mom_u_ff = r_mom_u_ff.loc["1994":]

#Create a new Data Frame
momentum_per_long = pd.DataFrame().rename_axis("1994-2022", axis=1)

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: R_momU_FF and UMD.
for data, column in zip(["Long and short, (R_mom_FF)", "Long only (R_momU_FF)"], list(r_mom_u_ff.columns)):
    momentum_per_long.loc[data, "Mean"] = r_mom_u_ff[column].mean()
    momentum_per_long.loc[data, "Vol"] = r_mom_u_ff[column].std()
    momentum_per_long.loc[data, "Sharpe"] = momentum_per_long.loc[data, "Mean"] / momentum_per_long.loc[data, "Vol"]
    momentum_per_long.loc[data, "Skewness"] = r_mom_u_ff[column].skew()
    momentum_per_long.loc[data, "Corr_to_Rm"] = r_mom_u_ff[column].corr(factors["MKT"])
    momentum_per_long.loc[data, "Corr_to_Rv"] = r_mom_u_ff[column].corr(factors["HML"])

momentum_per_long

1994-2022,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
"Long and short, (R_mom_FF)",0.009399,0.051645,0.181984,-0.482872,0.903499,-0.154667
Long only (R_momU_FF),0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601


### Task 4

Assess how sensitive the threshold for the 'winners' and 'losers' is in the results. Specifically, we compare three constructions:

- long the top 1 decile and short the bottom 1 deciles:

$\tilde{r}^{\text{momD1}} = r^{\text{mom(10)}}-r^{\text{mom(1)}}$

• long the top 3 deciles and short the bottom 3 deciles:

$\tilde{r}^{\text{momD3}} = 
(r^{\text{mom(8)}}+r^{\text{mom(9)}}+r^{\text{mom(10)}})/3-
(r^{\text{mom(1)}}+r^{\text{mom(2)}}+r^{\text{mom(3)}})/3 = 
\frac 13 \sum^{10}_{k=8} r^{\text{mom(k)}} + 
\frac 13 \sum^{3}_{k=1} r^{\text{mom(k)}}$
 

• long the top 5 deciles and short the bottom 5 deciles:

$\tilde{r}^{\text{momD5}} = 
\frac 15 \sum^{10}_{k=6} r^{\text{mom(k)}} + 
\frac 15 \sum^{5}_{k=1} r^{\text{mom(k)}}$

Compare all three constructions by filling out the stats in the table 3 below for the period 1994-2022.

<img src='table3.png'>

In [107]:
#import data from tables: deciles (total returns), and slice to the period 1994-2022
deciles = pd.read_excel("momentum_data.xlsx", "deciles (total returns)", index_col=0).loc["1994":]

#Create a new Data Frame
r_mom_d = pd.DataFrame()

#Create a list of our indicators
indexes_dec = list(deciles.columns)

#Calculate our indicators
r_mom_d["R_mom_D1"] = deciles[indexes_dec[-1]] - deciles[indexes_dec[0]]
r_mom_d["R_mom_D3"] = (deciles[indexes_dec[-3]] + deciles[indexes_dec[-2]] + deciles[indexes_dec[-1]]) / 3 \
- (deciles[indexes_dec[0]] + deciles[indexes_dec[1]] + deciles[indexes_dec[2]]) / 3
r_mom_d["R_mom_D5"] = (deciles[indexes_dec[-5]] + deciles[indexes_dec[-4]] +deciles[indexes_dec[-3]] \
                       + deciles[indexes_dec[-2]] + deciles[indexes_dec[-1]])/5 \
- (deciles[indexes_dec[0]] + deciles[indexes_dec[1]] + deciles[indexes_dec[2]] \
   + deciles[indexes_dec[3]] + deciles[indexes_dec[4]])/5

#Create a new Data Frame
momentum_constr = pd.DataFrame().rename_axis("1994-2022", axis=1)

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: R_mom_D1, R_mom_D3, R_mom_D5
for data in list(r_mom_d.columns):
    momentum_constr.loc[data, "Mean"] = r_mom_d[data].mean()
    momentum_constr.loc[data, "Vol"] = r_mom_d[data].std()
    momentum_constr.loc[data, "Sharpe"] = momentum_constr.loc[data, "Mean"] / momentum_constr.loc[data, "Vol"]
    momentum_constr.loc[data, "Skewness"] = r_mom_d[data].skew()
    momentum_constr.loc[data, "Corr_to_Rm"] = r_mom_d[data].corr(factors["MKT"])
    momentum_constr.loc[data, "Corr_to_Rv"] = r_mom_d[data].corr(factors["HML"])

momentum_constr

1994-2022,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
R_mom_D1,0.006617,0.086011,0.076936,-1.323613,-0.335574,-0.236392
R_mom_D3,0.002850,0.055590,0.051273,-1.355380,-0.365386,-0.229882
R_mom_D5,0.001503,0.038690,0.038837,-1.435409,-0.357936,-0.227501


### Task 5
Use the data provided on both small-stock 'winners', $r^{\text{momSU}}$, and small-stock 'losers', $r^{\text{momSD}}$
to construct a small-stock momentum portfolio,

$
r_t^{\text{momS}} = r^{\text{momSU}} - r^{\text{momSD}}
$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$
r_t^{\text{momB}} = r^{\text{momBU}} - r^{\text{momBD}}
$

Fill out Table 4 over the sample 1994-2022.

<img src='table4.png'>

In [108]:
#Create a new Data Frame
r_mom = pd.DataFrame()

#Calculate and add our indicators to Data Frame
r_mom["R_mom_FF"] = momentum["UMD"]
r_mom["R_mom_S"] = size_sorts["SMALL HiPRIOR"] - size_sorts["SMALL LoPRIOR"]
r_mom["R_mom_B"] = size_sorts["BIG HiPRIOR"] - size_sorts["BIG LoPRIOR"]

#Slice our data to the period of data: 1994-2022
r_mom = r_mom.loc["1994":]

#Create a new Data Frame
momentum_sm_bg = pd.DataFrame().rename_axis("1994-2022", axis=1)

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: 
#                                  All stocks(R_mom_FF), Small stocks (R_mom_S), Large stocks (R_mom_B)
for data, column in zip(["All stocks(R_mom_FF)", "Small stocks (R_mom_S)", "Large stocks (R_mom_B)"], list(r_mom.columns)):
    momentum_sm_bg.loc[data, "Mean"] = r_mom[column].mean()
    momentum_sm_bg.loc[data, "Vol"] = r_mom[column].std()
    momentum_sm_bg.loc[data, "Sharpe"] = momentum_sm_bg.loc[data, "Mean"] / momentum_sm_bg.loc[data, "Vol"]
    momentum_sm_bg.loc[data, "Skewness"] = r_mom[column].skew()
    momentum_sm_bg.loc[data, "Corr_to_Rm"] = r_mom[column].corr(factors["MKT"])
    momentum_sm_bg.loc[data, "Corr_to_Rv"] = r_mom[column].corr(factors["HML"])

momentum_sm_bg

1994-2022,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
All stocks(R_mom_FF),0.003946,0.048672,0.081075,-1.433554,-0.306139,-0.238601
Small stocks (R_mom_S),0.005228,0.049541,0.105531,-1.750109,-0.306908,-0.164672
Large stocks (R_mom_B),0.002660,0.052499,0.050667,-0.857615,-0.277984,-0.286919


### Task 6

Re-do Tables 2 and 4 but for the size portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- Size-sorted portfolios for value. "6 Portfolios Formed on Size and Book-to-Market (2 x 3)"

In [109]:
#import and clean data 
cmpt_me = pd.read_csv("6_Portfolios_2x3.CSV", header=9, index_col=0, nrows=1155, na_values=[-99.99, -999]).loc["1994":]

#Set the index Date Time to postnumerando
cmpt_me.index = pd.to_datetime(cmpt_me.index.astype("str"), format="%Y%m").to_period('M').to_timestamp('M')

cmpt_me.head()

,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
1994-01-31,1.8803,2.9469,4.8889,2.4154,4.5770,1.6973
1994-02-28,-0.9196,0.9023,-1.0985,-1.7641,-3.0804,-4.6720
1994-03-31,-6.2311,-4.7222,-4.3361,-4.6933,-3.9696,-3.3803
1994-04-30,-0.8882,0.7042,1.4197,0.6158,1.7767,1.6366
1994-05-31,-2.7048,-1.0815,0.1619,1.4041,1.5369,-0.1128


In [110]:
#Create a new Data Frame
r_mom_ff_cmpt = pd.DataFrame()

#Add indicator: R_mom_FF
r_mom_ff_cmpt["R_mom_FF"] = ((cmpt_me["BIG HiBM"] + cmpt_me["SMALL HiBM"])/2 \
- (cmpt_me["BIG LoBM"] + cmpt_me["SMALL LoBM"])/2)

r_mom_ff_cmpt

,R_mom_FF
1994-01-31,1.14525
1994-02-28,-1.54340
1994-03-31,1.60400
1994-04-30,1.66435
1994-05-31,0.67490
...,...
2022-05-31,8.41060
2022-06-30,-5.96580
2022-07-31,-4.09645
2022-08-31,0.30510


<img src='table2.png'>

In [111]:
#Create a new Data Frame
r_mom_u_ff_cmt = r_mom_ff_cmpt.copy()

#Add indicator: R_momU_FF
r_mom_u_ff_cmt["R_momU_FF"] = ((cmpt_me["BIG HiBM"] + cmpt_me["SMALL HiBM"])/2 - risk_free_rate["RF"]).loc["1994":]

#Create a new Data Frame
momentum_per_long_cmpt = pd.DataFrame()

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: R_momU_FF and UMD.
for data, column in zip(["Long and short, (R_mom_FF)", "Long only (R_momU_FF)"], list(r_mom_u_ff_cmt.columns)):
    momentum_per_long_cmpt.loc[data, "Mean"] = r_mom_u_ff_cmt[column].mean()
    momentum_per_long_cmpt.loc[data, "Vol"] = r_mom_u_ff_cmt[column].std()
    momentum_per_long_cmpt.loc[data, "Sharpe"] = momentum_per_long_cmpt.loc[data, "Mean"] / momentum_per_long_cmpt.loc[data, "Vol"]
    momentum_per_long_cmpt.loc[data, "Skewness"] = r_mom_u_ff_cmt[column].skew()
    momentum_per_long_cmpt.loc[data, "Corr_to_Rm"] = r_mom_u_ff_cmt[column].corr(factors.loc["1994":, "MKT"])
    momentum_per_long_cmpt.loc[data, "Corr_to_Rv"] = r_mom_u_ff_cmt[column].corr(factors.loc["1994":, "HML"])

momentum_per_long_cmpt

,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
"Long and short, (R_mom_FF)",0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Long only (R_momU_FF),0.998077,5.495839,0.181606,-0.908702,0.866864,0.330352


<img src='table4.png'>

In [112]:
#Create a new Data Frame
r_mom_FF_cmpt = r_mom_ff_cmpt.copy()

#Add indicator: R_mom_S, R_mom_B, and slice a data
r_mom_FF_cmpt["R_mom_S"] = (cmpt_me["SMALL HiBM"] - cmpt_me["SMALL LoBM"]).loc["1994":]
r_mom_FF_cmpt["R_mom_B"] = (cmpt_me["BIG HiBM"] - cmpt_me["BIG LoBM"]).loc["1994":]

#Create a new Data Frame
momentum_sm_bg_cmpt = pd.DataFrame().rename_axis("1994-2022", axis=1)

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: 
#                                  All stocks(R_mom_FF), Small stocks (R_mom_S), Large stocks (R_mom_B)
for data, column in zip(["All stocks(R_mom_FF)", "Small stocks (R_mom_S)", "Large stocks (R_mom_B)"], list(r_mom_FF_cmpt.columns)):
    momentum_sm_bg_cmpt.loc[data, "Mean"] = r_mom_FF_cmpt[column].mean()
    momentum_sm_bg_cmpt.loc[data, "Vol"] = r_mom_FF_cmpt[column].std()
    momentum_sm_bg_cmpt.loc[data, "Sharpe"] = momentum_sm_bg_cmpt.loc[data, "Mean"] / momentum_sm_bg_cmpt.loc[data, "Vol"]
    momentum_sm_bg_cmpt.loc[data, "Skewness"] = r_mom_FF_cmpt[column].skew()
    momentum_sm_bg_cmpt.loc[data, "Corr_to_Rm"] = r_mom_FF_cmpt[column].corr(factors["MKT"])
    momentum_sm_bg_cmpt.loc[data, "Corr_to_Rv"] = r_mom_FF_cmpt[column].corr(factors["HML"])

momentum_sm_bg_cmpt

1994-2022,Mean,Vol,Sharpe,Skewness,Corr_to_Rm,Corr_to_Rv
All stocks(R_mom_FF),0.132581,3.315845,0.039984,0.207751,-0.102250,1.000000
Small stocks (R_mom_S),0.306017,3.750472,0.081594,0.271015,-0.269082,0.900306
Large stocks (R_mom_B),-0.040855,3.641729,-0.011219,0.015334,0.090916,0.893837


### Task 7

Re-do Table 3 for the decile portfolios of the Value factor. Get this data from Ken French's website, https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html

Specifically, see the following data set:

- "Portfolios Formed on Book-to-Market"

In [113]:
#import and clean data 
portfolios = pd.read_csv("Portfolios_Formed_on_BE-ME.CSV", header=14, index_col=0, nrows=1155, na_values=[-99.99, -999]).loc["1994":].iloc[:,9:]

#Set the index Date Time to postnumerando
portfolios.index = pd.to_datetime(portfolios.index.astype("str"), format="%Y%m").to_period('M').to_timestamp('M')

portfolios

,Lo 10,Dec 2,Dec 3,Dec 4,Dec 5,Dec 6,Dec 7,Dec 8,Dec 9,Hi 10
1994-01-31,0.99,3.02,3.73,5.19,4.58,3.38,4.42,0.90,2.76,5.35
1994-02-28,-1.66,-1.62,-1.78,-0.41,-2.78,-2.71,-4.30,-4.32,-4.30,-2.39
1994-03-31,-5.27,-5.11,-3.99,-4.79,-3.59,-4.17,-3.94,-3.56,-2.38,-5.96
1994-04-30,0.65,0.66,0.07,1.16,1.57,2.34,1.63,1.39,1.60,2.13
1994-05-31,0.54,0.23,2.42,0.00,2.62,1.12,0.63,-1.68,1.87,0.43
...,...,...,...,...,...,...,...,...,...,...
2022-05-31,-3.22,-2.59,-1.38,2.46,2.45,3.37,6.99,4.50,6.76,5.94
2022-06-30,-8.01,-6.65,-5.74,-6.57,-8.76,-9.26,-11.19,-11.75,-11.81,-14.49
2022-07-31,14.33,8.09,7.05,5.64,6.07,4.86,6.07,9.27,7.38,6.27
2022-08-31,-5.09,-4.92,-2.99,-3.26,-0.83,-3.75,-0.22,-0.77,-2.82,-1.24


<img src='table3.png'>

In [114]:
#Create a new Data Frame
r_mom_d = pd.DataFrame()

#Calculate our indicators
r_mom_d["R_mom_D1"] = portfolios.iloc[:, -1] - portfolios.iloc[:, 0]
r_mom_d["R_mom_D3"] = portfolios.iloc[:, -3:].sum(axis=1) / 3 - portfolios.iloc[:, :3].sum(axis=1) / 3 
r_mom_d["R_mom_D5"] = portfolios.iloc[:, -5:].sum(axis=1) / 5 - portfolios.iloc[:, :5].sum(axis=1) / 5

#Create a new Data Frame
momentum_constr_por = pd.DataFrame().rename_axis("1994-2022", axis=1)

#Calculate "Mean, Vol, Sharpe, Skewness, Corr_to_Rm, Corr_to_Rv" for indicator: R_mom_D1, R_mom_D3, R_mom_D5
momentum_constr_por["Mean"], momentum_constr_por["Vol"], momentum_constr_por["Skewness"] = r_mom_d.mean(), r_mom_d.std(), r_mom_d.skew()
momentum_constr_por["Sharpe"] = momentum_constr_por["Mean"] / momentum_constr_por["Vol"]
for data in list(r_mom_d.columns):
    momentum_constr_por.loc[data, "Corr_to_Rm"] = r_mom_d[data].corr(factors["MKT"])
    momentum_constr_por.loc[data, "Corr_to_Rv"] = r_mom_d[data].corr(factors["HML"])

momentum_constr_por

1994-2022,Mean,Vol,Skewness,Sharpe,Corr_to_Rm,Corr_to_Rv
R_mom_D1,0.098754,5.106326,-0.152904,0.019339,0.178362,0.758798
R_mom_D3,0.048029,3.350795,-0.289519,0.014334,0.174831,0.819238
R_mom_D5,0.001467,2.373432,-0.039878,0.000618,0.158168,0.831363
